In [1]:
# Import the required modules
import cv2, os

import numpy as np

from PIL import Image

In [2]:
%ls /Users/guneevkaur/anaconda/pkgs/opencv-2.4.11-py27_1/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml

/Users/guneevkaur/anaconda/pkgs/opencv-2.4.11-py27_1/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml


In [3]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "/Users/guneevkaur/anaconda/pkgs/opencv-2.4.11-py27_1/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml"

#cascadePath="/Users/guneevkaur/anaconda/pkgs/opencv-2.4.11-py27_1/share/OpenCV/lbpcascades/lbpcascade_frontalface.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

In [4]:
# For face recognition we will the the LBPH Face Recognizer 
recognizer =cv2.face.LBPHFaceRecognizer_create()

In [5]:
 def get_images_and_labels(path):
    
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(path)
    
    #list to hold all subject faces
    images = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        #print dir_name
        label = int(dir_name.replace("s", ""))
        #print label
        #build path of directory containin images for current subject subject
        #sample subject_dir_path = "training-data/s1"
        subject_dir_path = path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        #print subject_images_names
        #------STEP-3--------
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
            
            
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name
            
            img = cv2.imread(image_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #height, width = img.shape[:2]
            #max_height = 300
            #max_width = 300

            # only shrink if img is bigger than required
            #if max_height < height or max_width < width:
            # get scaling factor
               # scaling_factor = max_height / float(height)
                   #if max_width/float(width) < scaling_factor:
                    #scaling_factor = max_width / float(width)
            # resize image
            #img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)


            #read image
            #display an image window to show the image 
            #cv2.imshow("Training on image...", cv2.resize(image, (400, 500)))
            #cv2.waitKey(100)
            # Convert the image format into numpy array
            image = np.array(gray, 'uint8')
        # Get the label of the image
            nbr = int(dir_name.replace("s", ""))
        # Detect the face in the image
            #print image
            faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
            for (x, y, w, h) in faces:
                images.append(image[y: y + h, x: x + w])
                labels.append(nbr)
                cv2.imshow("Adding faces to traning set...", image[y: y + h, x: x + w])
                cv2.waitKey(50)
    # return the images list and labels list
    return images, labels

In [6]:
import glob, os

for root, dirs, files in os.walk('/Users/guneevkaur/Desktop/datasets/training-data'):
    i = 0
    for file in files:
        if file.endswith('.DS_Store'):
            path = os.path.join(root, file)

            print "Deleting: %s" % (path)

            if os.remove(path):
                print "Unable to delete!"
            else:
                print "Deleted..."
                i += 1

print "Files Deleted: %d" % (i)

Files Deleted: 0


In [7]:
# Path to the  Dataset
path = "/Users/guneevkaur/Desktop/datasets/training-data"
# Call the get_images_and_labels function and get the face images and the 
# corresponding labels
images, labels = get_images_and_labels(path)
cv2.destroyAllWindows()


In [17]:
# Perform the tranining
recognizer.train(images, np.array(labels))

path="/Users/guneevkaur/Desktop/datasets/testing-data"
dirs = os.listdir(path)
for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
        
        subject_dir_path = path + "/" + dir_name
        subject_images_names = os.listdir(subject_dir_path)
        print subject_dir_path       
        for image_name in subject_images_names:
        # Read the image and convert to grayscale
            
            if image_name.startswith("."):
                continue;
                
            image_path = subject_dir_path + "/" + image_name   
# Append the images with the extension .sad into image_paths

            predict_image_pil = Image.open(image_path).convert('L')
            predict_image = np.array(predict_image_pil, 'uint8')
            faces = faceCascade.detectMultiScale(predict_image)
            
            for (x, y, w, h) in faces:
                
                nbr_predicted, conf = recognizer.predict(predict_image[y: y + h, x: x + w])
                
                nbr_actual = int(dir_name.replace("s", ""))
                if nbr_actual == nbr_predicted:
                    print "{} is Correctly Recognized with confidence {}".format(nbr_actual, conf)
                else:
                    print "{} is Incorrect Recognized as {}".format(nbr_actual, nbr_predicted)
                #cv2.imshow("Recognizing Face", predict_image[y: y + h, x: x + w])
                #cv2.waitKey(1000)

In [15]:
test_data_folder_path = "/Users/guneevkaur/Desktop/datasets/testing-data"
test_dirs = os.listdir(test_data_folder_path)
test_dirs.sort()
correct_pred = 0
test_subjects_count = 0

for directory in test_dirs:
    if directory == '.DS_Store':
        continue
    #print os.path.join(test_data_folder_path, directory, "1.png")
    subject_images_names = os.listdir(os.path.join(test_data_folder_path, directory))
    subject_images_names.sort()
    for img in subject_images_names:
        if not img.endswith("1.png"):
            continue
        test_img = cv2.imread(os.path.join(test_data_folder_path, directory, img))
        #print test_img
        gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
        test_label = recognizer.predict(gray)
        print directory.replace("s", "Person-"), "Person-{}".format(str(test_label[0]))
        if directory.replace("s", "Person-") == "Person-{}".format(str(test_label[0])):
            correct_pred += 1
        test_subjects_count += 1

Person-1 Person-6
Person-10 Person-6
Person-11 Person-12
Person-12 Person-12
Person-13 Person-17
Person-14 Person-17
Person-15 Person-9
Person-16 Person-16
Person-17 Person-21
Person-18 Person-21
Person-19 Person-19
Person-2 Person-6
Person-20 Person-23
Person-21 Person-21
Person-22 Person-23
Person-23 Person-23
Person-24 Person-23
Person-25 Person-21
Person-26 Person-21
Person-27 Person-21
Person-28 Person-8
Person-3 Person-6
Person-4 Person-6
Person-6 Person-7
Person-7 Person-6
Person-8 Person-8
Person-9 Person-9


In [16]:
print correct_pred*1.0/test_subjects_count

0.259259259259
